# 📓 Google Colab: Exploring Prompt Engineering Techniques with GPT-3.5 and GPT-4

## Introduction

This notebook demonstrates various prompt engineering techniques using OpenAI's GPT-3.5 or GPT-4. We explore techniques to improve generation performance, highlight failure cases, and how each method addresses these failures. The following approaches are illustrated:

- **In-Context Learning (ICL)**
- **Chain-of-Thought (CoT)**
- **Incremental Chain-of-Thought (iCOT)**
- **Tree-of-Thought (ToT)**
- **Graph-of-Thought (GoT)**
- **Agent-of-Thought (AoT)**
- **Reasoning with Self-Consistency and Error Feedback (RASCEF)**
- **Reasoning and Action Chain Extraction (REACT)**

In [3]:
## Setup
!pip install openai==0.28
import openai

# Set your OpenAI API key
openai.api_key = "OPENAI_API_KEY"

def get_response(system_prompt, user_message, model="gpt-4"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return response['choices'][0]['message']['content'].strip()

# Function to call the OpenAI API for simple prompts (like before)
def get_icl_response(icl_prompt, model="gpt-4"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "user", "content": icl_prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return response['choices'][0]['message']['content'].strip()


In [2]:
icl_prompt = """
Example 1: If Alice has 2 apples and Bob gives her 3 more apples, how many apples does Alice have?
Answer: 5

Example 2: If John has 5 bananas and gives 2 bananas to Sarah, how many bananas does John have left?
Answer: 3

Example 3: If Jane has 7 oranges and gives 4 to Emily, how many oranges does Jane have?
Answer:
"""
print("ICL Response:\n", get_icl_response(icl_prompt))


ICL Response:
 3


**Failure Case**
<br>
ICL often fails when there's insufficient data in the context or if the examples are ambiguous.

In [5]:
icl_failure_prompt = "Alice has an unknown number of apples, and Bob gives her some more. How many apples does Alice have now?"
print("ICL Failure Response:\n", get_icl_response(icl_failure_prompt))


ICL Failure Response:
 Without specific information on the number of apples Alice initially had and the number she received from Bob, it's impossible to determine the exact number of apples Alice has now.


## Chain-of-Thought (CoT)
**Example**
CoT encourages the model to generate intermediate reasoning steps to solve a problem.

In [7]:
cot_prompt = """
If Jack has 10 chocolates and eats 3 chocolates per day for 2 days, how many chocolates are left?

Step 1: Jack eats 3 chocolates per day.
Step 2: Jack eats 3 * 2 = 6 chocolates in 2 days.
Step 3: Initially, Jack has 10 chocolates.
Step 4: After eating 6 chocolates, Jack is left with 10 - 6 = 4 chocolates.
Answer: 4
"""

# Get response using ChatCompletion for CoT example
cot_response = get_icl_response(cot_prompt)
print("CoT Response:\n", cot_response)


CoT Response:
 Jack has 4 chocolates left.


**Failure Case** <br>
When steps are missing, the model can reach incorrect conclusions.

In [9]:
cot_failure_prompt = """
If Tom has 15 apples and eats 4 per day for 3 days, how many apples are left?
Answer without steps:
"""
print("CoT Failure Response:\n", get_icl_response(cot_failure_prompt))



CoT Failure Response:
 3


## Incremental Chain-of-Thought (iCOT)
Example
iCOT gradually reveals steps, which can lead to better logical consistency.

In [10]:
icot_prompt = """
Question: If Sarah has 18 candies and she gives 5 candies to her friend every day for 3 days, how many candies does she have left?

Step 1: How many candies does Sarah give in one day?
Answer: 5 candies.

Step 2: How many days does she give candies?
Answer: 3 days.

Step 3: Total candies given = 5 * 3 = 15.
Step 4: Remaining candies = 18 - 15 = 3.
Answer: 3
"""
print("iCOT Response:\n", get_icl_response(icot_prompt))


iCOT Response:
 Sarah has 3 candies left.


## Tree-of-Thought (ToT)
ToT helps handle complex questions by considering multiple branches of reasoning.

In [11]:
tot_prompt = """
Is it better to invest in stocks or bonds?
1. Consider risk levels:
    - Stocks: High risk, potentially high reward.
    - Bonds: Lower risk, stable returns.

2. Consider time horizon:
    - Long-term: Stocks may outperform.
    - Short-term: Bonds are safer.

Conclusion depends on risk appetite and time horizon.
"""
print("ToT Response:\n", get_icl_response(tot_prompt))


ToT Response:
 3. Consider economic conditions:
    - In a growing economy, stocks often perform better.
    - In a stagnant or declining economy, bonds may be safer.

4. Consider tax implications:
    - Some bonds, like municipal bonds, may be exempt from certain taxes.
    - Stocks may be subject to capital gains tax.

5. Consider diversification:
    - Holding a mix of both stocks and bonds can provide balance in your portfolio.

Ultimately, the decision to invest in stocks or bonds depends on your individual financial goals, risk tolerance, and investment strategy. It's often recommended to have a diversified portfolio that includes both stocks and bonds. It's also a good idea to consult with a financial advisor or do thorough research before making investment decisions.


## Graph-of-Thought (GoT)
Graph-of-Thought helps with complex relationships, often helpful in causal reasoning.

In [12]:
got_prompt = """
A -> B: Exercising improves health.
B -> C: Improved health leads to better mood.
D: Lack of sleep negatively affects mood.
If A and D happen, what is the effect on C?
"""
print("GoT Response:\n", get_icl_response(got_prompt))


GoT Response:
 The effect on C would be mixed. On one hand, exercising (A) would improve health (B) leading to a better mood (C). On the other hand, lack of sleep (D) would negatively affect the mood (C). So, the overall outcome would depend on the intensity and balance of these two effects.


## Agent-of-Thought (AoT)
Agents act as individual thinking entities providing input, enabling emergent reasoning.

In [13]:
aot_prompt = """
Agent 1: I think the answer is 10 based on my calculations.
Agent 2: I disagree. I think the answer should be recalculated considering extra factors.
Agent 3: Considering both, the adjusted answer might be 8.
Answer: 8
"""
print("AoT Response:\n", get_icl_response(aot_prompt))


AoT Response:
 Agent 1: That's interesting. I must have missed those factors. Can you clarify what they are?
Agent 2: Sure, the factors are X and Y, which have a significant impact on the result.
Agent 3: Correct, taking those into account changes the outcome.


## Reasoning with Self-Consistency and Error Feedback (RASCEF)
RASCEF allows multiple attempts, aiming for consistent answers by correcting errors.

In [14]:
rascef_prompt = """
If Tim has 25 dollars and spends 7 dollars each day, how many days can he spend before running out?

Attempt 1:
Day 1: 25 - 7 = 18.
Day 2: 18 - 7 = 11.
Day 3: 11 - 7 = 4.
Day 4: Not enough for a full 7.

Correct Answer: 3 full days.
"""
print("RASCEF Response:\n", get_icl_response(rascef_prompt))


RASCEF Response:
 Alternatively, you can divide the total amount of money Tim has by the amount he spends each day. 

25 / 7 = 3.57 

Since Tim cannot spend partial days, he can only spend for 3 full days.


## Reasoning and Action Chain Extraction (REACT)
REACT is useful when reasoning also involves taking actions.

In [15]:
react_prompt = """
User: Book a taxi to the airport.

Reasoning:
1. Find the user’s location.
2. Use a ride-hailing service.
3. Confirm time and location.

Action:
Booking a taxi for the requested time.
"""
print("REACT Response:\n", get_icl_response(react_prompt))


REACT Response:
 Assistant: Sure, I can help you with that. Can you please let me know the following details:
1. Your current location or pickup point.
2. The specific airport you are traveling to.
3. The date and time you would like the taxi to pick you up.
Once I have these details, I will be able to book the taxi for you.


# Conclusion
This notebook shows how prompt engineering techniques can make a difference in the responses from GPT models. Each technique has strengths and weaknesses, and understanding these helps in applying the correct approach for different types of problems.